In [2]:
from collections import namedtuple
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','done'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    


In [6]:
import torch
import torchvision
import gym

import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import gym
from gym.envs.registration import register
# register(
#    id='FrozenLakeNotSlippery-v0',
#    entry_point='gym.envs.toy_text:FrozenLakeEnv',
#    kwargs={'map_name' : '4x4', 'is_slippery': False},
#    max_episode_steps=100,
#    reward_threshold=0.78, # optimum = .8196
# )

#env = gym.make('FrozenLake8x8-v0')
#env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLakeNotSlippery-v0')
env.render()


use_cuda = torch.cuda.is_available
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

class q_net(nn.Module):
    def __init__(self):
        super(q_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4)
        

    def forward(self, x):
#         print("Q_Net: Input " + "-" *5)
#         print(x.shape)
#         print(x)
#         print("Q_Net: Input " + "-" *5)
        x = x.view(-1,1)
        x = F.tanh(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = F.tanh(self.linear2(x))
        x = x.view(-1,4)
        #print(x.shape)
        #print(x)
        return x
    
    


SFFF
FHFH
FFFH
HFFG


In [4]:
import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical
import random
random.seed(1999)
import math

# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    #print classname
    #print q_net
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        if not m.bias is None:
            m.bias.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)        
        #m.weight.data.fill_(0.1)
        #if not m.bias is None:
        #    m.bias.data.fill_(0.1)
        print m
        


# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mse = nn.MSELoss(reduce=False)
NUM_EPISODES = 1000000
BATCH_SIZE = 500
GAMMA = 0.9
TARGET_UPDATE = 50
EPS_START = 0.9
EPS_END = 0.0
EPS_DECAY = 1000000
online_net = q_net().to(device)
online_net.apply(weights_init)
target_net = q_net().to(device)
target_net.load_state_dict(online_net.state_dict())
target_net.eval()


memory = ReplayMemory(100000)
#optimizer = optim.RMSprop(online_net.parameters(), lr=0.001)
optimizer = optim.Adam(online_net.parameters(), lr=0.0001)

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    #non_final_mask = torch.tensor(tuple(map(lambda d: d is False,
    #                                      batch.done)), device=device, dtype=torch.uint8).unsqueeze(1)
    # Compute states that are final.
    next_state_final_mask = torch.tensor(tuple(map(lambda d: d in [5,7,11,12],
                                          batch.next_state)), device=device, dtype=torch.uint8).unsqueeze(1) 
    next_state_finak_list = [d for d in batch.next_state if d in [5,7,11,12] ]
    
    
    #non_final_next_states = torch.cat([FloatTensor([s]) for s,d in zip(batch.next_state,batch.done)
    #                                            if d is False])
    
    #state_batch = torch.cat([torch.FloatTensor([s]) for s in batch.state])
    state_batch = FloatTensor(batch.state)
    state_batch = state_batch.view(BATCH_SIZE, 1)
    next_state_batch = FloatTensor(batch.next_state)
    next_state_batch = next_state_batch.view(BATCH_SIZE, 1)
    #action_batch = torch.cat([torch.LongTensor([[a.item()]]) for a in batch.action])
    action_batch = LongTensor(batch.action).view(BATCH_SIZE,1)
    #reward_batch = torch.cat([torch.tensor([r]) for r in batch.reward])
    reward_batch = Tensor(batch.reward).view(BATCH_SIZE,1)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
#     print("state_batch "+"-" * 10)
#     print(state_batch.shape)
#     print("action_batch "+"-" * 10)
#     print(action_batch.shape)
    state_action_values = online_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # next_state_values = torch.zeros(BATCH_SIZE, device=device).view(BATCH_SIZE,1)
#     print("non_final_mask")
#     print(non_final_mask.shape)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # DDQN ------------------------
    # Y_ddqn = R_next + GAMMA * Q_target(S_next, argmax_a Q_online(S_next, a) )
    # Below is the argmax_a Q_online(S_next,a)
    argmax_a_online_net_next_state = online_net(next_state_batch).max(1)[1].detach().view(BATCH_SIZE,1)
    
    # Below is the actual Q_target
    next_state_values = target_net(next_state_batch).detach().gather(1, argmax_a_online_net_next_state)
    # DDQN ------------------------
#     print "next_state_values"
#     print next_state_values.shape
#     print next_state_values.type()
#     print "final_mask"
#     print final_mask.shape
#     print  final_mask.type()
#     print "next_state_values[final_mask]"
#     print next_state_values[final_mask].shape
#     print next_state_values[final_mask].type()
    next_state_values[next_state_final_mask] = torch.zeros(len(next_state_finak_list), device=device).view(len(next_state_finak_list))
#     print("next_state_values")
#     print(next_state_values.shape)
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     print("expected_state_action_values")
#     print(expected_state_action_values.shape)

    # Compute Huber loss (this is like MSE , but less sensitive to outliers )
    # loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    #loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
    loss = mse( state_action_values, expected_state_action_values)
    debug = False
    if debug:
        print("-" * 40)
        print("States")
        print(state_batch)
        print("Target Q")
        print(expected_state_action_values)
        print(expected_state_action_values.shape)
        print("Actual Q")
        print(state_action_values)
        print(state_action_values.shape)
        print("Loss")
        print(loss)
        print(loss.shape)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward(torch.ones(len(state_action_values), device=device).unsqueeze(1))
    
    # param in online_net.parameters():
    #    param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    if debug: 
        print("AFTER OPTIMIZATION:")
        print("New actual Q")
        print(online_net(state_batch).gather(1, action_batch))
    
    

Linear(in_features=1, out_features=20, bias=True)
Linear(in_features=20, out_features=4, bias=True)


In [9]:
score = []
times_trained = 0
times_reach_goal = 0
steps_done = 0


# TO BE DELETED_______
#steps_done = 40000000
#EPS_END = 0.0

# TO BE DELETED_______


for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    episode_series = []
    reward = 0
    episode_step = 0
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = FloatTensor(np_observation)
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        q_values = online_net(observation_tensor)
        if sample >= eps_threshold: 
            #print "observation_tensor"
            #print observation_tensor.type()
            
            action = q_values.max(1)[1] # First 1 is the dimension, second 1 is the index (this is argmax)
        else:
            action = torch.LongTensor([[random.randint(0,3)]], device=device)
        
            
        #break
        # Execute action in environment.
        old_state = observation                    
            
        
        observation, reward, done, info = env.step(action.item()) 
        new_state = observation
        
        

        
        
#         if done and reward != 1.0:
#             if episode_step > 50:
#                 reward = -0.2
#             if episode_step > 80:
#                 reward = -0.5
#             if new_state in [5,7,11,12]:
#                 reward = -1.0

        
        
        # Store the transition in memory
        #if old_state != new_state:

        memory.push(old_state, action, new_state, reward, done)
        if k%5000 == 0:
            #print("old_state != new_state")
            #print(old_state != new_state)
            #print("oldstate " + str(old_state) + " newstate " + str(new_state))
            print("q_values ")
            print(q_values)
            print("On state="+ str(old_state) + ", selected action=" + str(action.item()) + " , " + \
              "Random? " + str( sample < eps_threshold ))
            print("new state="+ str(new_state) + ", done="+str(done) + \
             ". Reward: " + str(reward))

        # Perform one step of the optimization (on the target network)
        if k > BATCH_SIZE :
            optimize_model()
            times_trained = times_trained + 1

        episode_step += 1
        #env.render()
    if k % TARGET_UPDATE == 0:
        target_net.load_state_dict(online_net.state_dict())
    
    if k % 1000 ==0:
        print_q_table()
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: \
              {}. Times reached goal: {}. \
              Steps done: {}. EPS_DECAY: {}. EPS_THRESHOLD: {}.".format(k, len(episode_series), \
                                                                    reward, np.mean(score), times_trained, \
                                                                       times_reach_goal, steps_done, \
                                                                       EPS_DECAY, eps_threshold))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        

    
    
    
    if reward > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    

q_values 
tensor(1.00000e-02 *
       [[ 4.6772,  1.5874, -1.5961,  1.0966]], device='cuda:0')
On state=0, selected action=0 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 4.6772,  1.5874, -1.5961,  1.0966]], device='cuda:0')
On state=0, selected action=2 , Random? True
new state=1, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 4.5986,  1.3899, -1.2254,  1.0709]], device='cuda:0')
On state=1, selected action=2 , Random? True
new state=2, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 4.5202,  1.1923, -0.8550,  1.0452]], device='cuda:0')
On state=2, selected action=3 , Random? True
new state=2, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 4.5202,  1.1923, -0.8550,  1.0452]], device='cuda:0')
On state=2, selected action=2 , Random? True
new state=3, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 4.4419,  0.9945, -0.4851,  1.0197]], device='cuda:0')
On state=3, selected

 state (0)  A[0]:(0.069325812161) A[1]:(0.078844152391) A[2]:(0.0393220558763) A[3]:(0.0706963539124)
 state (1)  A[0]:(0.075699865818) A[1]:(0.085556641221) A[2]:(0.0555234476924) A[3]:(0.072701536119)
 state (2)  A[0]:(0.0820658951998) A[1]:(0.0922584012151) A[2]:(0.0716966986656) A[3]:(0.0747012868524)
 state (3)  A[0]:(0.0884217917919) A[1]:(0.0989473238587) A[2]:(0.0878288596869) A[3]:(0.0766955837607)
 state (4)  A[0]:(0.0947655364871) A[1]:(0.105621322989) A[2]:(0.103907123208) A[3]:(0.0786844193935)
 state (5)  A[0]:(0.101095087826) A[1]:(0.112278327346) A[2]:(0.119918875396) A[3]:(0.0806677639484)
 state (6)  A[0]:(0.10740840435) A[1]:(0.118916288018) A[2]:(0.135851681232) A[3]:(0.082645624876)
 state (7)  A[0]:(0.113703519106) A[1]:(0.12553319335) A[2]:(0.151693403721) A[3]:(0.0846179947257)
 state (8)  A[0]:(0.119978442788) A[1]:(0.132127031684) A[2]:(0.167432203889) A[3]:(0.0865848809481)
 state (9)  A[0]:(0.126231238246) A[1]:(0.138695821166) A[2]:(0.183056563139) A[3]:(0.

 state (0)  A[0]:(0.0839584097266) A[1]:(0.100858271122) A[2]:(0.0432215332985) A[3]:(0.0845914110541)
 state (1)  A[0]:(0.0934173092246) A[1]:(0.111214406788) A[2]:(0.0663720369339) A[3]:(0.0879613310099)
 state (2)  A[0]:(0.102862127125) A[1]:(0.121547810733) A[2]:(0.089483641088) A[3]:(0.0913261175156)
 state (3)  A[0]:(0.112288765609) A[1]:(0.131853848696) A[2]:(0.112523272634) A[3]:(0.0946852862835)
 state (4)  A[0]:(0.12169316411) A[1]:(0.142127916217) A[2]:(0.135458216071) A[3]:(0.0980383455753)
 state (5)  A[0]:(0.131071299314) A[1]:(0.152365416288) A[2]:(0.158256158233) A[3]:(0.101384781301)
 state (6)  A[0]:(0.140419140458) A[1]:(0.162561878562) A[2]:(0.18088555336) A[3]:(0.104724124074)
 state (7)  A[0]:(0.149732753634) A[1]:(0.172712877393) A[2]:(0.203315749764) A[3]:(0.108055897057)
 state (8)  A[0]:(0.159008294344) A[1]:(0.182814031839) A[2]:(0.225517243147) A[3]:(0.111379608512)
 state (9)  A[0]:(0.168241903186) A[1]:(0.192861139774) A[2]:(0.247461676598) A[3]:(0.1146947

 state (0)  A[0]:(0.112252555788) A[1]:(0.128187283874) A[2]:(0.0632042512298) A[3]:(0.113169647753)
 state (1)  A[0]:(0.122474826872) A[1]:(0.137886807323) A[2]:(0.084853336215) A[3]:(0.115588977933)
 state (2)  A[0]:(0.132815122604) A[1]:(0.147727489471) A[2]:(0.107282154262) A[3]:(0.118247300386)
 state (3)  A[0]:(0.143262296915) A[1]:(0.157697796822) A[2]:(0.130420953035) A[3]:(0.121134974062)
 state (4)  A[0]:(0.153802856803) A[1]:(0.167783468962) A[2]:(0.154181465507) A[3]:(0.12423826009)
 state (5)  A[0]:(0.164421051741) A[1]:(0.177967444062) A[2]:(0.178457513452) A[3]:(0.127539142966)
 state (6)  A[0]:(0.17509894073) A[1]:(0.188230112195) A[2]:(0.203126862645) A[3]:(0.131015598774)
 state (7)  A[0]:(0.185816675425) A[1]:(0.19854940474) A[2]:(0.228053450584) A[3]:(0.134641692042)
 state (8)  A[0]:(0.19655276835) A[1]:(0.208901375532) A[2]:(0.253091037273) A[3]:(0.13838814199)
 state (9)  A[0]:(0.207284376025) A[1]:(0.219260394573) A[2]:(0.27808713913) A[3]:(0.142222866416)
 stat

 state (0)  A[0]:(0.176157608628) A[1]:(0.197418168187) A[2]:(0.199468404055) A[3]:(0.188027799129)
 state (1)  A[0]:(0.192781463265) A[1]:(0.217079609632) A[2]:(0.210899293423) A[3]:(0.19127163291)
 state (2)  A[0]:(0.207278996706) A[1]:(0.236557841301) A[2]:(0.166914969683) A[3]:(0.184860691428)
 state (3)  A[0]:(0.220786064863) A[1]:(0.257120370865) A[2]:(0.0875595733523) A[3]:(0.17233607173)
 state (4)  A[0]:(0.235638409853) A[1]:(0.280725240707) A[2]:(0.0260324515402) A[3]:(0.162502288818)
 state (5)  A[0]:(0.253617465496) A[1]:(0.308727383614) A[2]:(0.0269406344742) A[3]:(0.162597313523)
 state (6)  A[0]:(0.274859905243) A[1]:(0.341107964516) A[2]:(0.0963604599237) A[3]:(0.173651352525)
 state (7)  A[0]:(0.298526018858) A[1]:(0.376981586218) A[2]:(0.215389713645) A[3]:(0.192906230688)
 state (8)  A[0]:(0.323683857918) A[1]:(0.415256500244) A[2]:(0.358612805605) A[3]:(0.217295721173)
 state (9)  A[0]:(0.349616825581) A[1]:(0.454885870218) A[2]:(0.50305378437) A[3]:(0.24466343224)


...

 state (0)  A[0]:(0.532448291779) A[1]:(0.589743793011) A[2]:(0.590681433678) A[3]:(0.530992627144)
 state (1)  A[0]:(0.532675683498) A[1]:(0.000131949782372) A[2]:(0.656141042709) A[3]:(0.59034204483)
 state (2)  A[0]:(0.591494500637) A[1]:(0.728727340698) A[2]:(0.590435147285) A[3]:(0.655684173107)
 state (3)  A[0]:(0.656926095486) A[1]:(-0.0212405119091) A[2]:(0.532495737076) A[3]:(0.543586015701)
 state (4)  A[0]:(0.590882182121) A[1]:(0.655780851841) A[2]:(-0.000692129018717) A[3]:(0.530527949333)
 state (5)  A[0]:(-0.0273157190531) A[1]:(0.999896645546) A[2]:(-0.831160068512) A[3]:(0.674592018127)
 state (6)  A[0]:(-0.000255972146988) A[1]:(0.809886217117) A[2]:(-0.00120973528828) A[3]:(0.654800772667)
 state (7)  A[0]:(0.539698123932) A[1]:(-0.529891729355) A[2]:(0.441489011049) A[3]:(0.866142570972)
 state (8)  A[0]:(0.654651880264) A[1]:(0.00216017337516) A[2]:(0.728647947311) A[3]:(0.58815407753)
 state (9)  A[0]:(0.654081702232) A[1]:(0.810478925705) A[2]:(0.809706389904) A[

 state (0)  A[0]:(0.531245589256) A[1]:(0.590248227119) A[2]:(0.590510249138) A[3]:(0.531373739243)
 state (1)  A[0]:(0.53118622303) A[1]:(2.67773866653e-05) A[2]:(0.655946791172) A[3]:(0.590659976006)
 state (2)  A[0]:(0.590116977692) A[1]:(0.728897809982) A[2]:(0.590414762497) A[3]:(0.656259059906)
 state (3)  A[0]:(0.655987203121) A[1]:(-0.0207346007228) A[2]:(0.532608747482) A[3]:(0.544418871403)
 state (4)  A[0]:(0.590229094028) A[1]:(0.65579533577) A[2]:(-0.000214695930481) A[3]:(0.531702518463)
 state (5)  A[0]:(-0.0274378918111) A[1]:(0.999896705151) A[2]:(-0.830958545208) A[3]:(0.67604213953)
 state (6)  A[0]:(0.000287756323814) A[1]:(0.80989664793) A[2]:(-0.000413894624216) A[3]:(0.65612077713)
 state (7)  A[0]:(0.540283858776) A[1]:(-0.530588746071) A[2]:(0.442201793194) A[3]:(0.866740822792)
 state (8)  A[0]:(0.655919253826) A[1]:(0.000496461929288) A[2]:(0.728733897209) A[3]:(0.5910384655)
 state (9)  A[0]:(0.655745387077) A[1]:(0.810122072697) A[2]:(0.809895157814) A[3]:(

 state (0)  A[0]:(0.531782627106) A[1]:(0.590425372124) A[2]:(0.590409874916) A[3]:(0.531527996063)
 state (1)  A[0]:(0.531620979309) A[1]:(-8.3789229393e-05) A[2]:(0.656067728996) A[3]:(0.590474367142)
 state (2)  A[0]:(0.590593457222) A[1]:(0.729003548622) A[2]:(0.59060972929) A[3]:(0.656096637249)
 state (3)  A[0]:(0.656408667564) A[1]:(-0.0206706728786) A[2]:(0.532915472984) A[3]:(0.544050931931)
 state (4)  A[0]:(0.590609192848) A[1]:(0.655917286873) A[2]:(0.000186443328857) A[3]:(0.531324148178)
 state (5)  A[0]:(-0.0273588709533) A[1]:(0.999896764755) A[2]:(-0.830906927586) A[3]:(0.675995707512)
 state (6)  A[0]:(3.44216823578e-06) A[1]:(0.809998512268) A[2]:(0.000225424766541) A[3]:(0.655888676643)
 state (7)  A[0]:(0.539815545082) A[1]:(-0.530688226223) A[2]:(0.443028062582) A[3]:(0.866484880447)
 state (8)  A[0]:(0.655658125877) A[1]:(0.000107660889626) A[2]:(0.729053616524) A[3]:(0.590305089951)
 state (9)  A[0]:(0.655748724937) A[1]:(0.810020089149) A[2]:(0.810093045235) A[

 state (0)  A[0]:(0.531365096569) A[1]:(0.59057533741) A[2]:(0.590486347675) A[3]:(0.531560480595)
 state (1)  A[0]:(0.531381368637) A[1]:(-4.32431697845e-05) A[2]:(0.656120181084) A[3]:(0.589760124683)
 state (2)  A[0]:(0.590307891369) A[1]:(0.729106783867) A[2]:(0.590464711189) A[3]:(0.655285358429)
 state (3)  A[0]:(0.6563590765) A[1]:(-0.0167629215866) A[2]:(0.532871842384) A[3]:(0.543590068817)
 state (4)  A[0]:(0.591075778008) A[1]:(0.656341195107) A[2]:(0.00121593417134) A[3]:(0.531120121479)
 state (5)  A[0]:(-0.0269421990961) A[1]:(0.99989682436) A[2]:(-0.830863118172) A[3]:(0.676292181015)
 state (6)  A[0]:(0.00108396960422) A[1]:(0.810198545456) A[2]:(0.000406980485423) A[3]:(0.65626347065)
 state (7)  A[0]:(0.540988922119) A[1]:(-0.530084013939) A[2]:(0.442926317453) A[3]:(0.866585373878)
 state (8)  A[0]:(0.657454967499) A[1]:(0.000522285641637) A[2]:(0.729028761387) A[3]:(0.590941369534)
 state (9)  A[0]:(0.65848171711) A[1]:(0.810114264488) A[2]:(0.810219287872) A[3]:(0.

KeyboardInterrupt: 

In [8]:
def print_q_table():
    for i in range(16):
        st = np.array(i)
        st = np.expand_dims(st, axis=0)
        q_vals = online_net(FloatTensor(st))
        outp = " state (" +str(i) + ") "
        n = 0
        for tensr in q_vals:
            for cell in tensr:
                outp = outp + " A[" + str(n) + "]:(" + str(cell.item()) + ")"
                n += 1
        print(outp)
        
print_q_table()
print_q_table()

 state (0)  A[0]:(0.0467716865242) A[1]:(0.0158739089966) A[2]:(-0.015961188823) A[3]:(0.0109659750015)
 state (1)  A[0]:(0.0459862612188) A[1]:(0.0138985225931) A[2]:(-0.0122543023899) A[3]:(0.0107089262456)
 state (2)  A[0]:(0.045201510191) A[1]:(0.0119225373492) A[2]:(-0.00854972098023) A[3]:(0.0104518923908)
 state (3)  A[0]:(0.0444191098213) A[1]:(0.00994489248842) A[2]:(-0.00485114799812) A[3]:(0.0101966680959)
 state (4)  A[0]:(0.0436407215893) A[1]:(0.00796458125114) A[2]:(-0.00116223143414) A[3]:(0.00994502007961)
 state (5)  A[0]:(0.0428679808974) A[1]:(0.00598067371175) A[2]:(0.00251346104778) A[3]:(0.00969867687672)
 state (6)  A[0]:(0.0421024933457) A[1]:(0.0039923498407) A[2]:(0.0061724819243) A[3]:(0.00945930648595)
 state (7)  A[0]:(0.0413458012044) A[1]:(0.00199890718795) A[2]:(0.00981149822474) A[3]:(0.00922849774361)
 state (8)  A[0]:(0.0405994020402) A[1]:(-2.03028321266e-07) A[2]:(0.0134273516014) A[3]:(0.00900775659829)
 state (9)  A[0]:(0.0398647263646) A[1]:(-0.